# Defining the visual vocabulary

In [ ]:
import cv2 as cv
import numpy as np
import os
import sys

from vocabulary import Vocabulary, open_image, draw_keypoints

In [ ]:
dataset_dir = os.path.join('..', 'data')
n_words = 100

print(f'Creating vocabulary with {n_words} words')
sys.stdout.flush()
vocabulary = Vocabulary(n_words)

# Get relative paths for the images on the dataset directory
images = [os.path.join(dataset_dir, image) for image in sorted(os.listdir(dataset_dir))]
images = images[:500]

# Find keypoints for the vocabulary
vocabulary.train(images)

print(f'Vocabulary dimensions: {len(vocabulary.vocabulary.shape)}') 

In [ ]:
# Draw keypoints for the first image
query_image = images[0]

print('Processing first image...')
img_a = open_image(query_image)
if img_a is not None:
    detector = cv.BRISK_create()
    k, d = detector.detectAndCompute(img_a, None)
    words = []
    for i in range(d.shape[0]):
        words.append(vocabulary.which_word(d[i, :]))
    draw_keypoints('imageA', img_a, k, words)

# Training the Bag of Words vocabulary

In [ ]:
# Perform Vector Quantization to represent the histogram of features as a vector
# Vector Quantization assigns codes from a code book to observations
from scipy.cluster.vq import vq

img_features = np.zeros((len(images), n_words), 'float32')
for i in range(len(images)):
    words, distance = vq(vocabulary.descriptor_list[i][1], vocabulary.vocabulary)
    for word in words:
        img_features[i][word] += 1

In [ ]:
# Perform Tf-Idf Vectorization
n_occurences = np.sum((img_features > 0) * 1, axis=0)
idf = np.array(np.log((1.0 * len(images) + 1) / (1.0 * n_occurences + 1)), 'float32')

In [ ]:
# Scale the words (normalization)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(img_features)
img_features = scaler.transform(img_features)

In [ ]:
# Define classes for each image
img_class = []
with open('multiclass.csv') as img_classes:
    img_class = sorted(map(lambda line: line.strip().split(','), img_classes.readlines()))

classes = [data[1] for data in img_class]
classes = classes[:500]
print(classes[:10])

In [ ]:
# Train Support Vector Classification model to discriminate vectors
from sklearn.svm import LinearSVC
svc = LinearSVC(max_iter=50000) # 50000 iterations for better odds of converging
svc.fit(img_features, np.array(classes))

# Validating Bag of Words model